In [1]:
# imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression # Model
from sklearn.model_selection import cross_val_score, KFold # Model Evaluation
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore") # ignore, lol

In [3]:
train = pd.read_csv('Untitled Folder/train.csv')
test = pd.read_csv('Untitled Folder/test.csv') 
sam = pd.read_csv('Untitled Folder/sample_submission.csv')
print(train.shape)

(630000, 15)


In [9]:
train.isnull().sum()

id                         0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disease              0
dtype: int64

In [9]:
train.isnull().sum()

id                         0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disease              0
dtype: int64

In [10]:
train

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,56,0,1,110,226,0,0,132,0,0.0,1,0,7,Absence
629996,629996,54,1,4,128,249,1,2,150,0,0.0,2,0,3,Absence
629997,629997,67,1,4,130,275,0,0,149,0,0.0,1,2,7,Presence
629998,629998,52,1,4,140,199,0,2,157,0,0.0,1,0,6,Presence


In [11]:
test

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,630000,58,1,3,120,288,0,2,145,1,0.8,2,3,3
1,630001,55,0,2,120,209,0,0,172,0,0.0,1,0,3
2,630002,54,1,4,120,268,0,0,150,1,0.0,2,3,7
3,630003,44,0,3,112,177,0,0,168,0,0.9,1,0,3
4,630004,43,1,1,138,267,0,0,163,0,1.8,2,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269995,899995,58,1,2,120,222,0,0,172,0,1.0,1,0,7
269996,899996,58,1,4,132,289,0,0,172,0,2.8,2,0,3
269997,899997,63,1,3,108,201,1,0,158,0,0.8,1,0,3
269998,899998,59,1,4,120,274,0,2,163,0,0.5,1,0,3


In [12]:
train['Heart Disease']=train['Heart Disease'].map({'Absence':0, 'Presence':1})
train.drop('id', axis=1, inplace=True) # id is not used for model training and prediction
testID=test.pop('id') # important to preserve test IDs for final submission.
y=train.pop('Heart Disease') # TARGET

train.head(3)

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,58,1,4,152,239,0,0,158,1,3.6,2,2,7
1,52,1,1,125,325,0,2,171,0,0.0,1,0,3
2,56,0,2,160,188,0,2,151,0,0.0,1,0,3


In [13]:
train.nunique()

Age                         42
Sex                          2
Chest pain type              4
BP                          66
Cholesterol                150
FBS over 120                 2
EKG results                  3
Max HR                      93
Exercise angina              2
ST depression               66
Slope of ST                  3
Number of vessels fluro      4
Thallium                     3
dtype: int64

In [14]:
# One Hot All columns (Selective Encoding can be done,for now let's encode all)
oh_cols = train.columns.to_list()
oh = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_train = pd.DataFrame(oh.fit_transform(train[oh_cols]))
train[oh_train.columns.astype(str)] = oh_train.astype(int) #.astype('category')
oh_test= pd.DataFrame(oh.transform(test[oh_cols]))
test[oh_test.columns.astype(str)] = oh_test.astype(int) #.astype('category')

In [15]:
# Speed of Logistic Regression Increases 
# (results shall also increase, but speed is more observable)
scaler = StandardScaler()
scaled_train = pd.DataFrame(scaler.fit_transform(train))
scaled_train.columns = train.columns
scaled_test = pd.DataFrame(scaler.transform(test))
scaled_test.columns = test.columns

In [16]:
scaled_train.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,...,430,431,432,433,434,435,436,437,438,439
0,0.467921,0.631760,0.806994,1.435822,-0.178490,-0.294858,-0.982858,0.271190,1.628894,3.040655,...,-1.148335,1.207838,-0.158778,-1.556068,-0.452259,3.256005,-0.194028,-1.201903,-0.133097,1.246279
1,-0.258797,0.631760,-2.715729,-0.367088,2.374837,-0.294858,1.019582,0.951359,-0.613913,-0.754928,...,0.870826,-0.827926,-0.158778,0.642646,-0.452259,-0.307125,-0.194028,0.832014,-0.133097,-0.802388
2,0.225682,-1.582881,-1.541488,1.970017,-1.692672,-0.294858,1.019582,-0.095054,-0.613913,-0.754928,...,0.870826,-0.827926,-0.158778,0.642646,-0.452259,-0.307125,-0.194028,0.832014,-0.133097,-0.802388
3,-1.227755,-1.582881,-0.367247,0.233882,-0.475388,-0.294858,1.019582,-0.147375,-0.613913,0.299400,...,-1.148335,1.207838,-0.158778,0.642646,-0.452259,-0.307125,-0.194028,0.832014,-0.133097,-0.802388
4,0.467921,0.631760,0.806994,0.634529,-0.326939,-0.294858,1.019582,-1.455391,1.628894,3.251520,...,-1.148335,1.207838,-0.158778,-1.556068,-0.452259,-0.307125,5.153899,0.832014,-0.133097,-0.802388


In [17]:
model = LogisticRegression(max_iter=1000) # Our Model, simple one - Logistic Regression

In [18]:
kf = KFold(n_splits=5, shuffle=False)
i=0
oof=pd.Series(index=scaled_train.index)
test_p=[]
for train_index, val_index in kf.split(scaled_train):
    i+=1
    X_t, y_t = scaled_train.iloc[train_index], y.iloc[train_index]
    X_val, y_val = scaled_train.iloc[val_index], y.iloc[val_index]
    model.fit(X_t, y_t)
    
    temp=model.predict_proba(X_val)[:,-1]
    score = roc_auc_score(y_val, temp)
    print(f'Fold {i} \n Score: {score}')
    oof.iloc[val_index] = temp
    
    pred=model.predict_proba(scaled_test)[:,-1]
    test_p.append(pred)

print(f"CV: {roc_auc_score(y, oof)}")

Fold 1 
 Score: 0.9559832259453398
Fold 2 
 Score: 0.9551691957834394
Fold 3 
 Score: 0.9560222957084457
Fold 4 
 Score: 0.9548841345063634
Fold 5 
 Score: 0.9554397280122334
CV: 0.9555005396977433


In [19]:
out=sum(test_p)/5

final = pd.DataFrame({'id':testID, 'Heart Disease':out})
final.to_csv('submission.csv', index=False) # submission file'
print("Submitted Succesfully!!")
final.head()

Submitted Succesfully!!


,id,Heart Disease
0,630000,0.942312
1,630001,0.009032
2,630002,0.992439
3,630003,0.004610
4,630004,0.167358


In [21]:
oof.to_csv('oof_lr.csv', index=False)

In [23]:
print(x.columns)

NameError: name 'x' is not defined